## Audio model evaluation

In [ ]:
# load groundtruth (single label + multilabel), probabilities, one hot
# load as dataframe
import pandas as pd
from sklearn.metrics import cohen_kappa_score,f1_score,accuracy_score, precision_score, recall_score, classification_report, roc_auc_score, \
    hamming_loss
import numpy as np

LABELS_LIST = ['car', 'gym', 'happy', 'night', 'relax',
       'running', 'sad', 'summer', 'work', 'workout']

# [TODO] edit paths to match audio experiment output
exp_dir = "/src_code/repo/experiments_results/classic_updated_dataset_long/2020-05-08_13-17-43"
our_ground_truth = pd.read_csv(exp_dir+"/groundtruth_withIDS.csv")
our_predictions=  pd.read_csv(exp_dir+"/probabilities_withIDS.csv",)
one_hoted_df =  pd.read_csv(exp_dir+"/one_hoted_withIDS.csv")

## Single-output-single-groundtruth Protocol (SO-SG)

In [ ]:
## Per label evaluation of single-output-single-groundtruth 
# Create a dataframe where we keep all the evaluations
results_df = pd.DataFrame(columns=LABELS_LIST)
results_df.index.astype(str, copy=False)
percentage_of_positives_perclass = sum(our_ground_truth.values[:,2:]) / len(our_ground_truth)
results_df.loc[0] = percentage_of_positives_perclass
results_df.index = ['Ratio of positive samples']

# compute additional metrics (AUC,f1,recall,precision)
auc_roc_per_label = roc_auc_score(our_ground_truth.values[:,2:], our_predictions.values[:,2:], average=None)
precision_perlabel = precision_score(our_ground_truth.values[:,2:], one_hoted_df.values[:,2:], average=None)
recall_perlabel = recall_score(our_ground_truth.values[:,2:], one_hoted_df.values[:,2:], average=None)
f1_perlabel = f1_score(our_ground_truth.values[:,2:], one_hoted_df.values[:,2:], average=None)

results_df = results_df.append(
    pd.DataFrame([auc_roc_per_label,recall_perlabel, precision_perlabel, f1_perlabel], columns=LABELS_LIST))
results_df.index = ['Ratio of positive samples',"AUC", "Recall", "Precision", "f1-score"]
results_df['average'] = results_df.mean(numeric_only=True, axis=1)
results_df.round(3).T

## Multi-output-single-groundtruth Protocol (MO-SG)

In [ ]:
"""
    Evaluate on multi-label output and single-label grountruth
"""
# Create a dataframe where we keep all the evaluations
our_predictions=  pd.read_csv(exp_dir+"/probabilities_withIDS.csv",)
model_output_rounded = np.round(our_predictions.values[:,2:])
model_output_rounded = np.clip(model_output_rounded, 0, 1)
results_df = pd.DataFrame(columns=LABELS_LIST)
results_df.index.astype(str, copy=False)
percentage_of_positives_perclass = sum(our_ground_truth.values[:,2:]) / len(our_ground_truth)
results_df.loc[0] = percentage_of_positives_perclass
results_df.index = ['Ratio of positive samples']

# compute additional metrics (AUC,f1,recall,precision)
auc_roc_per_label = roc_auc_score(our_ground_truth.values[:,2:], our_predictions.values[:,2:], average=None)
precision_perlabel = precision_score(our_ground_truth.values[:,2:], model_output_rounded, average=None)
recall_perlabel = recall_score(our_ground_truth.values[:,2:], model_output_rounded, average=None)
f1_perlabel = f1_score(our_ground_truth.values[:,2:], model_output_rounded, average=None)

results_df = results_df.append(
    pd.DataFrame([auc_roc_per_label,recall_perlabel, precision_perlabel, f1_perlabel], columns=LABELS_LIST))
results_df.index = ['Ratio of positive samples',"AUC", "Recall", "Precision", "f1-score"]
results_df['average'] = results_df.mean(numeric_only=True, axis=1)
results_df.round(3).T
# get plots of confusion matrix